収集したデータから一重と八重のフォルダに画像を分ける

ディレクトリ構成
8_9/
|- /a - 'a-01.jpeg', 'a-02.jpeg', ... , 'a-64.jpeg' 
|- /b - 'b-01.jpeg', 'b-02.jpeg', ... , 'b-64.jpeg' 
|- /c ...
|- /d ... 
|- /e ...
|- /f   .
|- /g   .
|- /h   .
|- /i   .
|- /j   .
|- /k   .
|- /l   .
|- /m   .
|- /n   .
|- /o   .
|- /p - 'p-01.jpeg', 'p-02.jpeg', ... , 'p-64.jpeg' 
|- /single
|- /double
|- /dead
|- /not_boomed
|- a_p.csv
|- labeling.ipynb

In [23]:
import os
import re
import glob
import shutil
import pandas as pd

目的 
/a ~ /p に入っている画像データをa_p.csvのラベルに従って、/single, /double, /dead, /not_bloomed
に画像を移動させること。これらのディレクトリはそれぞれ順番に、一重、八重、枯れてしまった、まだ開花していない
を意味している。

In [24]:
def make_4_dir():
    flag = False
    dir_list = glob.glob('*')
    if ('single' in dir_list) == False:
        os.makedirs('single')
        print('This probram made \' single \'directory. ')
        flag = True
    if ('double' in dir_list) == False:
        os.makedirs('double')
        print('This probram made \' double \'directory. ')
        flag = True
    if ('dead' in dir_list) == False:
        os.makedirs('dead')
        print('This probram made \' dead \'directory. ')
        flag = True
    if ('not_bloomed' in dir_list) == False:
        os.makedirs('not_bloomed')
        print('This probram made \' not_bloomed \'directory. ')
        flag = True

    if flag==False:
        print('All directory has already exist.')


In [25]:
!pwd
csv_file = 'a_p.csv'
df = pd.read_csv(csv_file)
df

/Users/machidakentarou/Documents/test/datas/8_12


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
0,4,4.0,8.0,4,4,NaN,1.0,1,1,8.0,4.0,4,4,8.0,4.0,8
1,8,4.0,1.0,8,8,1.0,8.0,4,8,8.0,1.0,4,8,8.0,8.0,8
2,1,4.0,4.0,4,4,8.0,1.0,4,8,NaN,NaN,1,8,1.0,1.0,1
3,8,4.0,1.0,4,4,8.0,8.0,4,1,8.0,8.0,1,8,8.0,NaN,8
4,1,1.0,8.0,4,8,1.0,4.0,8,8,1.0,1.0,8,1,8.0,1.0,4
5,4,8.0,8.0,8,8,8.0,4.0,4,1,1.0,4.0,8,4,8.0,1.0,4
6,4,8.0,4.0,8,1,1.0,8.0,8,8,4.0,4.0,4,8,8.0,8.0,8
7,4,1.0,4.0,4,4,4.0,4.0,1,4,4.0,8.0,8,8,8.0,1.0,8
8,4,8.0,8.0,8,4,4.0,1.0,4,1,8.0,8.0,4,4,8.0,8.0,8
9,1,1.0,NaN,1,4,8.0,4.0,4,4,8.0,4.0,4,1,8.0,NaN,4


ラベルはこのようについている。1が一重、8が八重、4が枯れてしまった芽、NaNがまだ開花していない芽
をそれぞれ表している。

In [26]:
def make_dataset(dir_alpabet_list,csv_file):
    natural_alp = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p']
    #natural_alp = ['A','B','C','D','E','F','G','H','Q','R','S','T','U','V','W','X']
    # natural_alp = ['aa','ab','ac','ad','ae','af','ag','ah','ai','aj','ak','al','am','an','ao','ap']
    for dir_alpabet,natural in zip(dir_alpabet_list,natural_alp):
        images_2d_list = []
        target_dir = './' + dir_alpabet + '/*'
        files = glob.glob(target_dir)
        files.sort()

        for i in range(16):
            images_2d_list.append(files[i::16])

        df = pd.read_csv(csv_file)

        single_path = 'single/'
        double_path = 'double/'
        dead_path = 'dead/'
        not_bloomed_path = 'not_bloomed/'
        judge_list = df[natural]

        # 画像に欠けがあったら移動を行わない。
        if(len(files)%16 != 0 or len(files)==0):
            print('The number of images in folder ' + dir_alpabet + ' is not multiple of 16.')
        # ディレクトリの数が16でなければ移動を行わない。
        elif(len(dir_alpabet_list)!=16):
            print('The number of directory is not 16.')
        else:
            for img_list, judge in zip(images_2d_list,judge_list):
                for p in img_list:
                    if judge == 8:
                        try:
                            shutil.copy(p,double_path)
                        except:
                            print(p,'has already moved!')
                    elif judge == 1:
                        try:
                            shutil.copy(p,single_path)
                        except:
                            print(p,'has already moved!')
                    elif judge == 4:
                        try:
                            shutil.copy(p,dead_path)
                        except:
                            print(p,'has already moved!')
                    else:
                        try:
                            shutil.copy(p,not_bloomed_path)
                        except:
                            print(p,'has not bloomed yet')

In [27]:
make_4_dir()
dir_list = glob.glob('*')
dir_list.remove('double')
dir_list.remove('single')
dir_list.remove('dead')
dir_list.remove('not_bloomed')
dir_list.remove('labeling.ipynb')
dir_list.remove(csv_file)
dir_list.sort() # アルファベット順に並べ替える必要がある
print('directory list : ',dir_list)
make_dataset(dir_list,csv_file)

All directory has already exist.
directory list :  ['a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2', 'i2', 'j2', 'k2', 'l2', 'm2', 'n2', 'o2', 'p2']
The number of images in folder j2 is not multiple of 16.


In [28]:
double_list = glob.glob('./double/*')
print('The number of double is ',len(double_list))
single_list = glob.glob('./single/*')
print('The number of double is ',len(single_list))
print(100*len(double_list)/(len(single_list) + len(double_list)))

The number of double is  341
The number of double is  246
58.09199318568995
